In [4]:
import pandas as pd
import numpy as np
import re
import seaborn
import faker
from faker import Faker
from random import sample
########Focusing on the Impact of Mail Campign##############

In [33]:
####Sample Size####
n = 100
x = 0.5 # percentage to take for Mail Campign
m = 50
####Adjust the Price and Units to watch the different impact####
Mail_StartDate, Mail_EndaDate, Mail_price, Mail_Unit,Mail_Bor = '02/01/2018','03/01/2018', 10,m,'Manhattan'

#####Create Campign info table to record all the campagin details####
data = np.array([['Campign_Name','StartDate','EndDate','Price','Units','Borough'],
                ['TV','01/01/2018','04/01/2018', 5,n,'Manhattan'],
                ['Mail',Mail_StartDate, Mail_EndaDate, Mail_price, m, Mail_Bor],
                ['Train_Ads','04/01/2018','04/01/2018', 12,300,'Manhattan']]) 

campign_info = pd.DataFrame(data = data[1:,:],
                  columns = data[0,:])


,Campign_Name,StartDate,EndDate,Price,Units,Borough
0,TV,01/01/2018,04/01/2018,5,100,Manhattan
1,Mail,02/01/2018,03/01/2018,10,50,Manhattan
2,Train_Ads,04/01/2018,04/01/2018,12,300,Manhattan


In [41]:
####Create Resident Info, when they are affected by campagin, GrubHub enrollment status and enrollment Start date####
class Resident:
  def __init__(self): #, Enrollment, Enrollment_StartDatem, Mail_Target, Train_Target):
    self.fake = Faker()
    self.FirstName = self.fake.first_name()
    self.LastName = self.fake.last_name()
    self.Address = self.fake.street_name()
    self.PhoneNumber = self.fake.phone_number()
 
resident_info = pd.DataFrame(columns =['FirstName','LastName','Address','Borough','PhoneNumer','Enrollment',
                                       'Enrollment_StartDate','TV_Target','Mail_Target','Train_Target'])

for i in range(n):
        resident = Resident()
        resident_info.loc[i,:4] = [resident.FirstName,resident.LastName,resident.Address,resident.PhoneNumber]
        resident_info.loc[i,'TV_Target'] = True


rindex =  np.array(sample(range(len(resident_info)), m))
resident_info.loc[rindex,'Mail_Target'] = True
resident_info[resident_info['Mail_Target'] == True]

,FirstName,LastName,Address,PhoneNumer,Enrollment,Enrollment_StartDate,TV_Target,Mail_Target,Train_Target
2,Katherine,Miller,Amy Valley,04583803016,NaN,NaN,True,True,NaN
3,Ricardo,Sanders,Kelsey Vista,08953607987,NaN,NaN,True,True,NaN
5,Jillian,Pierce,Howard Tunnel,07443806633,NaN,NaN,True,True,NaN
8,Ashley,Harvey,Amanda Trail,06753870590,NaN,NaN,True,True,NaN
11,Blake,Schroeder,Emily Walk,234-193-1197x62903,NaN,NaN,True,True,NaN
14,Gina,Blake,Jonathan Course,1-996-623-3924,NaN,NaN,True,True,NaN
15,Kristina,Jackson,Jennifer Center,045-468-6937,NaN,NaN,True,True,NaN
21,Holly,Garcia,Horn Fields,136.271.1187x7941,NaN,NaN,True,True,NaN
24,Kristin,Warren,Clayton Field,05054847820,NaN,NaN,True,True,NaN
27,Courtney,Bailey,Davis Road,223-920-1331x008,NaN,NaN,True,True,NaN


In [42]:
####Simulate the TV Campign and Mail Campign####
resident_info

,FirstName,LastName,Address,PhoneNumer,Enrollment,Enrollment_StartDate,TV_Target,Mail_Target,Train_Target
0,Jessica,Gonzales,Frank Street,(726)560-1861x6687,NaN,NaN,True,NaN,NaN
1,Mike,Fernandez,Gallagher Cove,(365)804-0452x743,NaN,NaN,True,NaN,NaN
2,Katherine,Miller,Amy Valley,04583803016,NaN,NaN,True,True,NaN
3,Ricardo,Sanders,Kelsey Vista,08953607987,NaN,NaN,True,True,NaN
4,Andrew,Hull,Ramos Island,+31(6)2591501780,NaN,NaN,True,NaN,NaN
5,Jillian,Pierce,Howard Tunnel,07443806633,NaN,NaN,True,True,NaN
6,Justin,Shelton,George Pike,1-153-550-9277,NaN,NaN,True,NaN,NaN
7,Joshua,Daugherty,Wong Bypass,1-099-109-6802x94940,NaN,NaN,True,NaN,NaN
8,Ashley,Harvey,Amanda Trail,06753870590,NaN,NaN,True,True,NaN
9,Lucas,Mendez,Johnson Corners,08897655972,NaN,NaN,True,NaN,NaN
